# Addressing the Problem

### Skin Disease Classification

The U.S. Census Bureau projects that by 2050, about half of all patients seen in the U.S. healthcare system will have skin of color (SOC). As these communities continue to grow, so will the likelihood of dermatologists encountering cutaneous (skin) diseases that happen more frequently in SOC patients, happen exclusively in SOC patients, and/or present differently in SOC patients than their White counterparts. Aiding dermatologists in their understanding, and more importantly diagnoses, of SOC patients’ cutaneous disease presentations is paramount to delivering life-saving
quality of care to these communities. As a result, the following practicum project will explore the use of computer vision for classification of cutaneous diseases in SOC patients.

### Problem Structure

The following research notebook addresses the supervised classification problem of determining whether a given image of a skin disease is benign (0) or malignant (1). The metric(s) to be used for evaluating the neural network's performance is TBD.

### Import Necessary Modules

In [1]:
# Data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split

# Neural network libraries
import tensorflow as tf
from keras import models, layers, utils

# AWS libraries
import boto3

2024-04-05 01:33:00.145583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Retrieval

My dataset of choice is the Diverse Dermatology Images (DDI) dataset curated by the Stanford University School of Medicine. As can be seen below, we begin by fetching the metadata CSV from the appropriate S3 location. The data workflow pipeline has already run the metadata transform Glue job on the raw metadata prior to the execution of this notebook, so we're merely fetching the cleaned product. However, further feature engineering will be performed on the metadata and images in preparation for model fitting.

Upon further inspection of the metadata, we can also see that it contains 656 fully-populated records, two of which being integer features and the other two being string features.

### Data Fetch

In [2]:
s3 = boto3.client('s3')
bucket = 'poc-skin-disease-detection-cv'

In [3]:
data_file_key = 'ddidiversedermatologyimages/metadata/transform'
data_file_name = 'part-00000-f5ce70f1-c572-4cbc-818e-e8dda587c3ef-c000.csv'

obj = s3.get_object(Bucket=bucket, Key='{}/{}'.format(data_file_key, data_file_name)) 
ddi_df = pd.read_csv(obj['Body'])

In [4]:
ddi_df.head()

,DDI_file,skin_tone,malignant,disease
0,000001.png,56,1,melanoma-in-situ
1,000002.png,56,1,melanoma-in-situ
2,000003.png,56,1,mycosis-fungoides
3,000004.png,56,1,squamous-cell-carcinoma-in-situ
4,000005.png,12,1,basal-cell-carcinoma


### Data Inspection

In [5]:
ddi_df.shape

(656, 4)

In [6]:
ddi_df.dtypes

DDI_file     object
skin_tone     int64
malignant     int64
disease      object
dtype: object

In [7]:
ddi_df.isnull().sum()

DDI_file     0
skin_tone    0
malignant    0
disease      0
dtype: int64

In [8]:
ddi_df.describe()

,skin_tone,malignant
count,656.000000,656.000000
mean,33.966463,0.260671
std,17.511578,0.439336
min,12.000000,0.000000
25%,12.000000,0.000000
50%,34.000000,0.000000
75%,56.000000,1.000000
max,56.000000,1.000000


# Exploratory Data Analysis (EDA)

### Which cutaneous diseases comprise our dataset?

The initial scope of this project was to specifically assess melanomas. However, a total of 78 cutaneous disease presentations, including other well-known cutaneous conditions such as carcinomas, have been aggregated to form the DDI dataset.

In [9]:
print("Number of disease presentations: ", len(ddi_df['disease'].unique()))
np.sort(ddi_df['disease'].unique())

Number of disease presentations:  78


array(['abrasions-ulcerations-and-physical-injuries', 'abscess',
       'acne-cystic', 'acquired-digital-fibrokeratoma',
       'acral-melanotic-macule', 'acrochordon', 'actinic-keratosis',
       'angioleiomyoma', 'angioma', 'arteriovenous-hemangioma',
       'atypical-spindle-cell-nevus-of-reed', 'basal-cell-carcinoma',
       'basal-cell-carcinoma-nodular', 'basal-cell-carcinoma-superficial',
       'benign-keratosis', 'blastic-plasmacytoid-dendritic-cell-neoplasm',
       'blue-nevus', 'cellular-neurothekeoma', 'chondroid-syringoma',
       'clear-cell-acanthoma', 'coccidioidomycosis',
       'condyloma-accuminatum', 'dermatofibroma', 'dermatomyositis',
       'dysplastic-nevus', 'eccrine-poroma',
       'eczema-spongiotic-dermatitis', 'epidermal-cyst',
       'epidermal-nevus', 'fibrous-papule', 'focal-acral-hyperkeratosis',
       'folliculitis', 'foreign-body-granuloma', 'glomangioma',
       'graft-vs-host-disease', 'hematoma', 'hyperpigmentation',
       'inverted-follicular-k

### Which skin tones are present in this dataset?

What makes the DDI dataset unique is its inclusion of skin types across the skin tone spectrum. The skin tone index values below are representations of the Fitzpatrick Skin Types (FST): fairer skin tones (Types 1-2), medium skin tones (Types 3-4), and darker skin tones (Types 5-6). Given the record counts associated with each FST, we can conclude that the dataset samples are relatively balanced.

In [10]:
ddi_df.drop(['DDI_file', 'malignant'], axis=1).groupby(['skin_tone']).count()

,disease
skin_tone,
12,208
34,241
56,207


### What is the dataset's target variable distribution?

As was mentioned in the opening section, a skin disease presentation can be classified as either benign (0) or malignant (1); this is our dataset's target variable. Based on the data presented below, we can see that approximately 1 in 4 (26.1%) presentations will be malignant, a probability we should keep in mind when evaluating the classifications generated by the selected neural network. Additionally, we can see the classes are relatively balanced in their distribution across the FSTs, so no data resampling should be necessary.

In [11]:
pd.concat([ddi_df['malignant'].value_counts(), ddi_df['malignant'].value_counts(normalize=True)], axis=1)

,count,proportion
malignant,,
0,485,0.739329
1,171,0.260671


In [12]:
ddi_df.drop('DDI_file', axis=1).groupby(['malignant', 'skin_tone']).count()

disease
malignant skin_tone         
0         12             159
          34             167
          56             159
1         12              49
          34              74
          56              48

# Feature Engineering & Analysis

### Image Augmentation

In [13]:
%%sh
mkdir -p images

In [14]:
def download_s3_images(images_file_key):  
    for image_file_name in ddi_df['DDI_file'].tolist():    
        output_path = 'images/{}'.format(image_file_name)
        
        with open(output_path, 'wb') as image_output:
            s3.download_fileobj(
                Bucket=bucket, 
                Key='{}/{}'.format(images_file_key, image_file_name),
                Fileobj=image_output
            )

In [15]:
download_s3_images(images_file_key='ddidiversedermatologyimages/images/raw')

In [16]:
images = utils.image_dataset_from_directory(
    directory='images/',
    labels=ddi_df['malignant'].tolist(),
    label_mode='binary',
    image_size=(512, 512),
    shuffle=False,
    # pad_to_aspect_ratio=True,
)

Found 656 files belonging to 2 classes.


### Data Preprocessing

In [17]:
X = ddi_df.drop(['malignant'], axis=1)
y = ddi_df['malignant']

# Model Construction & Training

# Model Evaluation

# Future Work